In [41]:
-- Gloria Budiman
-- EE569 HW4
-- 6528-1836-50
-- 11/25/2016

require 'nn'
require 'optim'

torch.manualSeed(0)
torch.setnumthreads(4)

In [42]:
function setup() 
    classes = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 }
    geometry = { 32, 32 }

    net = nn.Sequential()

    net:add(nn.SpatialConvolution(1, 6, 5, 5))
    net:add(nn.ReLU())
    net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

    net:add(nn.SpatialConvolution(6, 16, 5, 5))
    net:add(nn.ReLU())
    net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

    net:add(nn.View(16*5*5))
    net:add(nn.Linear(16*5*5, 120))
    net:add(nn.ReLU())
    net:add(nn.Linear(120, 84))
    net:add(nn.ReLU())
    net:add(nn.Linear(84, #classes))
    net:add(nn.LogSoftMax())
    
    parameters, gradParameters = net:getParameters()
    criterion = nn.ClassNLLCriterion()
    confusion = optim.ConfusionMatrix(classes)
end

In [43]:
function random_translate(obj) 
    n = torch.Tensor(1, 40, 40):zero()
    n[{ {1}, {5, 36}, {5, 36 }}] = obj
    
    rand_x = torch.random(-6, 6)
    rand_y = torch.random(-6, 6)
    
    n = image.translate(n, rand_x, rand_y)
    n = image.crop(n, 4, 4, 36, 36)
    return n
end

In [44]:
function normalize(input, n_channels)
    local mean = {}
    local stdev = {}
    
    for channel = 1, n_channels do
        mean[channel] = input.data[{ {}, {channel}, {}, {} }]:mean()
        stdev[channel] = input.data[{ {}, {channel}, {}, {} }]:std()
        
        print('Channel ' .. channel .. ' mean: ' .. mean[channel] .. ' stdev: ' .. stdev[channel])
        
        input.data[{ {}, {channel}, {}, {} }]:add(-mean[channel])
        input.data[{ {}, {channel}, {}, {} }]:div(stdev[channel])
    end
end

In [45]:
function load_data()    
    train = torch.load('mnist-p1b-train.t7')
    test = torch.load('mnist-p1b-test.t7')
    
    n_train = train.data:size()[1]
    n_test = test.data:size()[1]
    
    for i = 1, n_test do
        test.data[i] = random_translate(test.data[i])
    end
    
    train.data = train.data:double()
    test.data = test.data:double()
end

In [46]:
function exec_training(obj, n_channels, current_epoch) 
    confusion:zero()
    
    for t = 1, n_train, batch_size do
        local limit = math.min(t + batch_size - 1, n_train)
        
        local limited_batch_size = limit - t + 1
        local inputs = torch.Tensor(limited_batch_size, n_channels, geometry[1], geometry[2])
        local targets = torch.Tensor(limited_batch_size)
        local k = 1
        
        for i = t, limit do
            local input = obj.data[i]:clone()
            local target = obj.label[i]
            
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end
        
        local feval = function(x)
            collectgarbage()
            
            if x ~= parameters then
                parameters:copy(x)
            end
            gradParameters:zero()
            
            local outputs = net:forward(inputs)
            local f = criterion:forward(outputs, targets)
            local df_do = criterion:backward(outputs, targets)
            net:backward(inputs, df_do)
            
            for i = 1, limited_batch_size do
                confusion:add(outputs[i], targets[i])
            end
            
            return f, gradParameters
        end
        
        sgd_state = sgd_state or {
            learningRate = 0.03,
            learningRateDecay = 1e-7,
            momentum = 0.5
        }
        optim.sgd(feval, parameters, sgd_state)
    end
    
    confusion:updateValids()
    return confusion.averageValid
end

In [47]:
function exec_testing(obj, n_channels, current_epoch)
    confusion:zero()
    
    for t = 1, n_test, batch_size do
        local limit = math.min(t + batch_size - 1, n_test)
        
        local limited_batch_size = limit - t + 1
        local inputs = torch.Tensor(limited_batch_size, n_channels, geometry[1], geometry[2])
        local targets = torch.Tensor(limited_batch_size)
        local k = 1
        
        for i = t, limit do
            local input = obj.data[i]:clone()
            local target = obj.label[i]
            
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end
        
        local preds = net:forward(inputs)
        
        for i = 1, limited_batch_size do
            confusion:add(preds[i], targets[i])
        end
    end 
    
    confusion:updateValids()
    return confusion.averageValid
end

In [49]:
setup()
load_data()
normalize(train, 1)
normalize(test, 1)

n_epoch = 32
batch_size = 256
local last_train_conf
local last_test_conf
for epoch = 1, n_epoch do
    local acc_train = exec_training(train, 1, epoch)
    
    if (epoch == n_epoch) then
        last_train_conf = confusion:__tostring__()
    end
    
    local acc_test = exec_testing(test, 1, epoch)
    
    if (epoch == n_epoch) then
        last_test_conf = confusion:__tostring__()
    end
    
    io.write(string.format('Epoch %3d: %.4f%% | %.4f%%\n', epoch, acc_train * 100, acc_test * 100))
end

print(last_train_conf)
print(last_test_conf)

Channel 1 mean: 25.509416422526 stdev: 70.180423838273	


Channel 1 mean: 25.78058671875 stdev: 70.643073560573	


Epoch   1: 61.5291% | 23.8881%


Epoch   2: 91.3684% | 27.2515%


Epoch   3: 94.8469% | 30.8610%


...

Epoch  28: 99.4845% | 40.9736%


Epoch  29: 99.5169% | 40.8764%


Epoch  30: 99.5328% | 40.9341%


Epoch  31: 99.5868% | 40.9842%


Epoch  32: 99.5977% | 40.9668%
ConfusionMatrix:
[[    5917       1       0       0       1       0       3       0       0       1]   99.899% 	[class: 1]
 [       1    6723       0       0       0       0       2      11       3       2]   99.718% 	[class: 2]
 [       2       3    5947       1       1       0       1       2       1       0]   99.815% 	[class: 3]
 [       1       0       4    6099       0       7       0       2      15       3]   99.478% 	[class: 4]
 [       0       3       0       0    5822       0       2       1       2      12]   99.658% 	[class: 5]
 [       1       1       0       8       0    5395       8       1       7       0]   99.520% 	[class: 6]
 [       5       0       1       0       3       5    5902       0       2       0]   99.730% 	[class: 7]
 [       0       9       4       2       4       0       0    6241       1       4]   99.617% 	[class: 8]
 [       2       4       5      10       1       7       4       3    5809       6]   99.282% 	[class: 9